### 1. Importing all the needed librries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import sys
!{sys.executable} -m pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 14.6MB/s ta 0:00:01
     |████████████████████████████████| 92kB 13.4MB/s eta 0:00:01
Libraries imported.


### 2. Scrap the data from wikipedia  to a pandas dataframe

In [2]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Hyderabad,_India").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
# create a new DataFrame from the list
#kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
hyd_df = pd.DataFrame({"Neighborhood": neighborhoodList})

# print the number of rows of the dataframe
hyd_df.shape

(54, 1)

In [5]:
# go through all the neighborhoods of Hyderabad once to check if it has all the data.
hyd_df.head(54)

,Neighborhood
0,"► Abids‎ (1 C, 13 F)"
1,"► Alwal‎ (1 C, 1 F)"
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)"
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)"
4,"► Banjara Hills‎ (3 C, 25 F)"
5,"► Basheerbagh‎ (1 C, 7 F)"
6,"► Begumpet‎ (5 C, 9 F)"
7,► Boduppal‎ (2 F)
8,"► Bolarum‎ (3 C, 1 F)"
9,"► Cavalry Barracks, Hyderabad‎ (1 C)"


### 3. Get the lat long of the neighborhoods listed above

In [6]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in hyd_df["Neighborhood"].tolist() ]

In [8]:
coords

[[17.389800000000037, 78.47658000000007],
 [17.535430000000076, 78.54427000000004],
 [17.43482000000006, 78.44949000000008],
 [17.299820000000068, 78.46495000000004],
 [17.415350000000046, 78.43435000000005],
 [17.40211000000005, 78.47770000000008],
 [17.447290000000066, 78.45396000000005],
 [17.40954000000005, 78.57896000000005],
 [17.536214413606626, 78.23504495152397],
 [17.4089385, 78.32673900000002],
 [17.40301000000005, 78.49792000000008],
 [17.4089385, 78.32673900000002],
 [17.368570000000034, 78.53515000000004],
 [17.409950000000038, 78.48229000000003],
 [17.45333000000005, 78.43034000000006],
 [17.43181000000004, 78.38636000000008],
 [17.522760000000062, 78.43862000000007],
 [17.466849, 78.24916574999999],
 [17.389410000000055, 78.40406000000007],
 [17.32707000000005, 78.60533000000004],
 [17.429230000000075, 78.37495000000007],
 [17.399230000000045, 78.48073000000005],
 [17.36838000000006, 78.39999000000006],
 [17.42865000000006, 78.39762000000007],
 [17.386880000000076, 78.4

In [9]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [10]:
# merge the coordinates into the original dataframe
hyd_df['Latitude'] = df_coords['Latitude']
hyd_df['Longitude'] = df_coords['Longitude']

In [11]:
# check the neighborhoods and the coordinates
print(hyd_df.shape)
hyd_df

(54, 3)


,Neighborhood,Latitude,Longitude
0,"► Abids‎ (1 C, 13 F)",17.389800,78.476580
1,"► Alwal‎ (1 C, 1 F)",17.535430,78.544270
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",17.434820,78.449490
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",17.299820,78.464950
4,"► Banjara Hills‎ (3 C, 25 F)",17.415350,78.434350
5,"► Basheerbagh‎ (1 C, 7 F)",17.402110,78.477700
6,"► Begumpet‎ (5 C, 9 F)",17.447290,78.453960
7,► Boduppal‎ (2 F),17.409540,78.578960
8,"► Bolarum‎ (3 C, 1 F)",17.536214,78.235045
9,"► Cavalry Barracks, Hyderabad‎ (1 C)",17.408939,78.326739


In [12]:
# save the DataFrame as CSV file
hyd_df.to_csv("hyd_df.csv", index=False)

### 4.Hyderabad Map with Neighborhoods superimposed

In [13]:
# get the coordinates of Hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The latlong of Hyderabad, India is{}, {}.'.format(latitude, longitude))

The latlong of Hyderabad, India is17.38878595, 78.46106473453146.


In [14]:
# create map of Hyderabad using latitude and longitude values
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hyd)  
    
map_hyd

In [15]:
# save the map as HTML file
map_hyd.save('map_hyd.html')

### 5. Explore the neighbors using Foursquare APIs

In [16]:
# define Foursquare Credentials and Version
CLIENT_ID = 'SCS3WPIR5LYRLPSI2S4CF0MZNS1ZGAOXINCKOBHJL3IB05KW' 
CLIENT_SECRET = '3JJKKS0JBQIXY3VVG4WA5PSKQDVJ52DMFLEU3YZAJ45YGFF0' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SCS3WPIR5LYRLPSI2S4CF0MZNS1ZGAOXINCKOBHJL3IB05KW
CLIENT_SECRET:3JJKKS0JBQIXY3VVG4WA5PSKQDVJ52DMFLEU3YZAJ45YGFF0


In [17]:
# Define the radius and limit for accessing the foursquare APIs

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2164, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Pragati,17.388088,78.481134,South Indian Restaurant
1,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Mayur Pan Shop,17.388894,78.480578,Juice Bar
2,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Santosh Dhaba,17.388485,78.479509,Indian Restaurant
3,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Taj Mahal Hotel,17.391942,78.476915,Hotel
4,"► Abids‎ (1 C, 13 F)",17.3898,78.47658,Karachi Bakery,17.383454,78.475075,Bakery


In [19]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Abids‎ (1 C, 13 F)",79,79,79,79,79,79
"► Alwal‎ (1 C, 1 F)",4,4,4,4,4,4
"► Ameerpet, Hyderabad‎ (3 C, 21 F)",100,100,100,100,100,100
"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",4,4,4,4,4,4
"► Banjara Hills‎ (3 C, 25 F)",100,100,100,100,100,100
"► Basheerbagh‎ (1 C, 7 F)",98,98,98,98,98,98
"► Begumpet‎ (5 C, 9 F)",50,50,50,50,50,50
► Boduppal‎ (2 F),7,7,7,7,7,7
"► Bolarum‎ (3 C, 1 F)",3,3,3,3,3,3


In [20]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 152 uniques categories.


In [21]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:25]

array(['South Indian Restaurant', 'Juice Bar', 'Indian Restaurant',
       'Hotel', 'Bakery', 'Ice Cream Shop', 'Shoe Store', 'Food Truck',
       'Neighborhood', 'Chaat Place', 'Diner', 'Lounge', 'Burger Joint',
       'Dessert Shop', 'Café', 'Snack Place', 'Science Museum',
       'Chinese Restaurant', 'Stadium', 'Restaurant', 'Coffee Shop',
       'Smoke Shop', 'Fast Food Restaurant', 'Breakfast Spot',
       'Department Store'], dtype=object)

In [22]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyzing each neighborhods by taking the mean of  the frequency of occurance

In [23]:
# one hot encoding
hyd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hyd_onehot.columns[-1]] + list(hyd_onehot.columns[:-1])
hyd_onehot = hyd_onehot[fixed_columns]

print(hyd_onehot.shape)
hyd_onehot.head()

(2164, 153)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racetrack,Rajasthani Restaurant,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Women's Store
0,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"► Abids‎ (1 C, 13 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
hyd_grouped = hyd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(hyd_grouped.shape)
hyd_grouped

(52, 153)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racetrack,Rajasthani Restaurant,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Women's Store
0,"► Abids‎ (1 C, 13 F)",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.037975,0.012658,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.012658,0.012658,0.000000,0.050633,0.012658,0.050633,0.000000,0.012658,0.000000,0.025316,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.012658,0.037975,0.012658,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.012658,0.063291,0.000000,0.012658,0.000000,0.012658,0.000000,0.012658,0.00,0.012658,0.00,0.000000,0.012658,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012658,0.037975,0.012658,0.000000,0.000000,0.000000,0.063291,0.139241,0.000000,0.012658,0.00,0.000000,0.000000,0.025316,0.000000,0.000000,0.000000,0.00,0.012658,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.012658,0.00,0.000000,0.00,0.012658,0.012658,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.012658,0.000000,0.025316,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.037975,0.000000,0.000000,0.000000,0.012658,0.012658,0.012658,0.012658,0.025316,0.000000,0.00,0.025316,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000
1,"► Alwal‎ (1 C, 1 F)",0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000

In [25]:
len(hyd_grouped[hyd_grouped["Shopping Mall"] > 0])

20

In [26]:
hyd_mall = hyd_grouped[["Neighborhoods","Shopping Mall"]]

In [27]:
hyd_mall.head(20)

,Neighborhoods,Shopping Mall
0,"► Abids‎ (1 C, 13 F)",0.012658
1,"► Alwal‎ (1 C, 1 F)",0.000000
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.020000
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000
5,"► Basheerbagh‎ (1 C, 7 F)",0.010204
6,"► Begumpet‎ (5 C, 9 F)",0.000000
7,► Boduppal‎ (2 F),0.000000
8,"► Bolarum‎ (3 C, 1 F)",0.000000
9,"► Cavalry Barracks, Hyderabad‎ (1 C)",0.000000


### 7. Using KNN method to cluster into 3 segments

In [28]:
# set number of clusters
kclusters = 3

kl_clustering = hyd_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 0, 2, 2, 0, 0, 0, 0], dtype=int32)

In [29]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hyd_merged = hyd_mall.copy()

# add clustering labels
hyd_merged["Cluster Labels"] = kmeans.labels_

In [30]:
hyd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hyd_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"► Abids‎ (1 C, 13 F)",0.012658,2
1,"► Alwal‎ (1 C, 1 F)",0.000000,0
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.020000,2
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000,0
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000,2


In [31]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hyd_merged = hyd_merged.join(hyd_df.set_index("Neighborhood"), on="Neighborhood")

print(hyd_merged.shape)
hyd_merged.head() # check the last columns!

(52, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"► Abids‎ (1 C, 13 F)",0.012658,2,17.38980,78.47658
1,"► Alwal‎ (1 C, 1 F)",0.000000,0,17.53543,78.54427
2,"► Ameerpet, Hyderabad‎ (3 C, 21 F)",0.020000,2,17.43482,78.44949
3,"► Bandlaguda, Rangareddy‎ (1 C, 2 F)",0.000000,0,17.29982,78.46495
4,"► Banjara Hills‎ (3 C, 25 F)",0.020000,2,17.41535,78.43435


In [32]:
# sort the results by Cluster Labels
print(hyd_merged.shape)
hyd_merged.sort_values(["Cluster Labels"], inplace=True)
hyd_merged

(52, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
51,"► Trimulgherry‎ (1 C, 3 F)",0.000000,0,17.470719,78.504503
44,► Sanathnagar‎ (8 F),0.000000,0,17.458760,78.443100
21,► Hydershakote‎ (14 F),0.000000,0,17.368380,78.399990
43,► Pedda Amberpet‎ (1 F),0.000000,0,17.321150,78.642370
23,"► Kachiguda‎ (1 C, 4 F)",0.000000,0,17.386880,78.495530
50,"► Tarnaka‎ (1 C, 6 F)",0.000000,0,17.408939,78.326739
27,► L. B. Nagar‎ (16 F),0.000000,0,17.512650,78.441290
37,"► Nacharam‎ (1 C, 4 F)",0.000000,0,17.433510,78.566730
28,"► Madhapur‎ (1 C, 19 F)",0.000000,0,17.458598,78.368032
30,"► Malkajgiri‎ (3 C, 6 F)",0.000000,0,17.439137,78.529172


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighborhood'], hyd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
# save the map as HTML file
map_clusters.save('map_clusters_Hyd.html')

In [35]:

hyd_merged.loc[hyd_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
51,"► Trimulgherry‎ (1 C, 3 F)",0.0,0,17.470719,78.504503
44,► Sanathnagar‎ (8 F),0.0,0,17.458760,78.443100
21,► Hydershakote‎ (14 F),0.0,0,17.368380,78.399990
43,► Pedda Amberpet‎ (1 F),0.0,0,17.321150,78.642370
23,"► Kachiguda‎ (1 C, 4 F)",0.0,0,17.386880,78.495530
50,"► Tarnaka‎ (1 C, 6 F)",0.0,0,17.408939,78.326739
27,► L. B. Nagar‎ (16 F),0.0,0,17.512650,78.441290
37,"► Nacharam‎ (1 C, 4 F)",0.0,0,17.433510,78.566730
28,"► Madhapur‎ (1 C, 19 F)",0.0,0,17.458598,78.368032
30,"► Malkajgiri‎ (3 C, 6 F)",0.0,0,17.439137,78.529172


In [36]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
17,"► Golconda‎ (5 C, 2 F)",0.071429,1,17.389410,78.404060
12,"► Dilsukhnagar‎ (1 C, 2 F)",0.047619,1,17.368570,78.535150
36,"► Moula-Ali‎ (3 C, 5 F)",0.058824,1,17.465770,78.560180
26,► Kukatpally‎ (16 F),0.090909,1,17.487350,78.420870
38,"► Nagole, Hyderabad‎ (4 F)",0.071429,1,17.372418,78.544476


In [37]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
42,"► Old City (Hyderabad, India)‎ (8 C, 26 F)",0.010309,2,17.408438,78.478441
49,► Somajiguda‎ (5 F),0.020000,2,17.420720,78.463000
0,"► Abids‎ (1 C, 13 F)",0.012658,2,17.389800,78.476580
35,► Moazzam Jahi Market‎ (15 F),0.037037,2,17.384480,78.474420
32,► Masab Tank‎ (4 F),0.010000,2,17.400930,78.453620
24,"► Khairtabad‎ (1 C, 2 F)",0.020000,2,17.405920,78.458560
22,"► Jubilee Hills‎ (3 C, 8 F)",0.010000,2,17.428650,78.397620
20,► Hyderguda‎ (2 F),0.010989,2,17.399230,78.480730
18,"► HITEC City‎ (5 C, 29 F)",0.038462,2,17.429230,78.374950
15,"► Gachibowli‎ (4 C, 17 F)",0.010000,2,17.431810,78.386360
